In [1]:
#from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

Load the Nashville Zip Code data & Nashville Business Improvement District data

In [2]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head( )

EPSG:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


Now read in the capital improvements data.

In [3]:
capital_imp = pd.read_csv('../data/NDOT_Capital_Projects.csv')
print(capital_imp.shape)
capital_imp.head(3)

(477, 32)


,Project ID,Organization,Department,Division,Project Number,Project Category,Project Type,Project Sub-Type,Phase,Priority,...,Actual Cost,Metro Cost,State Cost,Federal Cost,Other Cost,Total Cost,Longitude,Latitude,Layer Name,Mapped Location
0,7120,Metro Nashville,Metro Public Works,Engineering,21-060710410857,Bikeway,Protected Bike Lane,New,90% Plan Design,NaN,...,0.00,0.00,0.0,0,0,227000.00,-86.777767,36.164337,Project Line,POINT (-86.777767 36.164337)
1,8332,Metro Nashville,Metro Public Works,Engineering,2023-R-4,Capital,Intersection Improvements,Rehabilitation,Planning/Scoping,NaN,...,0.00,0.00,0.0,0,0,0.00,-86.680802,36.026929,Project Poly,POINT (-86.680802 36.026929)
2,5070,Metro Nashville,Metro Public Works,Engineering,2016-R-041,Sidewalk,New Sidewalk,New,Construction/Closeout/Warranty,NaN,...,358547.67,2057712.98,0.0,0,0,358547.67,-86.778107,36.247520,Project Line,POINT (-86.778107 36.24752)


In [4]:
capital_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Project ID        477 non-null    int64  
 1   Organization      477 non-null    object 
 2   Department        477 non-null    object 
 3   Division          477 non-null    object 
 4   Project Number    468 non-null    object 
 5   Project Category  477 non-null    object 
 6   Project Type      477 non-null    object 
 7   Project Sub-Type  477 non-null    object 
 8   Phase             477 non-null    object 
 9   Priority          0 non-null      float64
 10  Status            477 non-null    object 
 11  Project Name      477 non-null    object 
 12  Description       477 non-null    object 
 13  Justification     477 non-null    object 
 14  Project Location  477 non-null    object 
 15  Notes             475 non-null    object 
 16  Council District  477 non-null    object 
 1

In [5]:
zipcodes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Create GeoDataFrame

In [6]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]
capital_imp= capital_imp[['Department', 'Division', 'Project Category', 'Project Type', 'Phase', 'Total Cost', 'Longitude', 'Latitude']]
capital_geo = gpd.GeoDataFrame(capital_imp, crs= zipcodes.crs, geometry=gpd.points_from_xy(capital_imp.Longitude, capital_imp.Latitude))
type(capital_geo)

geopandas.geodataframe.GeoDataFrame

Combine with a spatial join

In [7]:
projects_by_zip = gpd.sjoin(capital_geo, zipcodes, predicate='within')
projects_by_zip.head()

,Department,Division,Project Category,Project Type,Phase,Total Cost,Longitude,Latitude,geometry,index_right,zip,po_name
0,Metro Public Works,Engineering,Bikeway,Protected Bike Lane,90% Plan Design,227000.00,-86.777767,36.164337,POINT (-86.77777 36.16434),36,37201,NASHVILLE
8,Metro Public Works,Engineering,Sidewalk,Sidewalk Repairs,Complete,680796.79,-86.774708,36.161951,POINT (-86.77471 36.16195),36,37201,NASHVILLE
9,Metro Public Works,Engineering,Sidewalk,Sidewalk Repairs,Complete,850143.79,-86.774843,36.162175,POINT (-86.77484 36.16217),36,37201,NASHVILLE
51,Metro Public Works,Engineering,Sidewalk,Sidewalk Repairs,Complete,795675.79,-86.775514,36.161615,POINT (-86.77551 36.16161),36,37201,NASHVILLE
61,Metro Public Works,Engineering,Sidewalk,New Sidewalk,Construction/Closeout/Warranty,1500000.00,-86.776494,36.161196,POINT (-86.77649 36.16120),36,37201,NASHVILLE


How many projects are there of each type?

In [10]:
projects_by_zip['Project Type'].value_counts()

Project Type
New Sidewalk                 132
Bike Lane                     68
Paving                        63
Bridge Repairs                38
Shared Bikeway                36
Protected Bike Lane           26
Intersection Improvements     25
Sidewalk Repairs              20
Guardrail Repairs             14
Buffered Bike Lane            10
Fall Paving                    6
Bridge Replacement             6
Culvert Replacement            5
New Guardrail                  5
Information Technology         5
Special Ops                    5
Roadway Improvements           4
New Traffic Signal             2
Replace Bridge Deck            2
New Bridge                     2
New Pedestrian Signal          1
Summer Paving                  1
Culvert Repairs                1
Name: count, dtype: int64

New sidewalk cnstruction is the most common project type. Look at these projects by zip code.